# Einfaches Machine Learning mit `scikit-learn`

`scikit-learn` ist das Standard-Tool für Machine Learning in Python. Fast alle *klassischen ML-Verfahren* werden durch `scikit-learn` unterstützt.

`scikit-learn` verfügt (im Gegensatz zu vielen anderen Open-Source-Paketen) über eine fantatische Dokumentation, die du dir unter https://scikit-learn.org/stable/ unbedingt anschauen solltest.

Sehr praktisch ist auch, dass `scikit-learn` bereits Datensets mitbringt, mit denen man die Funktionalität ausprobieren kann.

## Supervised Learning

Als ganz einfaches Beispiel nutzen wir hier ein Datenset, das aus 20 Newsgroups besteht. Es ist zwar schon ziemlich alt, aber die Sprache hat sich nicht sehr geändert. Das Datenset lässt sich ganz leicht laden:

In [1]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')

Die Daten findest du in dem Schlüssel `data` des zurückgelieferten `dict`.

In [2]:
len(newsgroups_train['data'])

11314

Und jetzt kannst du fast schon mit dem Machine Learning beginnen. Die Daten müssen nur noch *vektorisiert* werden. Was dabei genau passiert, schauen wir uns später noch an:

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(newsgroups_train.data)

Sobald du die Vektoren zur Verfügung hast, kannst du einen Klassifikator trainieren, hier eine sog. *Support Vector Machine*. Der Klassifikator soll lernen, aufgrund eines Newsgroups-Posting die korrekte Kategorie (in diesem Fall die Newsgroup, in der gepostet wurde) zu erraten. Ein solches Problem heißt *Klassifikationsproblem*.

In [4]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3, random_state=42)
clf.fit(vectors, newsgroups_train.target)

SGDClassifier(random_state=42)

Das ging **schnell**. Immerhin waren es über 11.000 Texte, aus denen das Modell die Kategoriezuordnung gelernt hat.

Anhand einer unabhängigen Datenmenge kannst die Ergebnisse verfizieren:

In [5]:
from sklearn import metrics
newsgroups_test = fetch_20newsgroups(subset='test')
vectors_test = vectorizer.transform(newsgroups_test.data)
pred = clf.predict(vectors_test)
metrics.f1_score(newsgroups_test.target, pred, average='macro')

0.8445789829167119

Auch das geh sehr schnell und mit über 84% *F-Score* ziemlich genau. Wenn du dir anschauen möchtest, wo Fehler passiert sind, hilft die sog. *Confusion Matrix*, die in der Diagonale die richtigen Ergebnisse zeigt:

In [6]:
import pandas as pd
pd.DataFrame(metrics.confusion_matrix(newsgroups_test.target, pred), 
             columns=newsgroups_train.target_names, index=newsgroups_train.target_names)

,alt.atheism,comp.graphics,comp.os.ms-windows.misc,comp.sys.ibm.pc.hardware,comp.sys.mac.hardware,comp.windows.x,misc.forsale,rec.autos,rec.motorcycles,rec.sport.baseball,rec.sport.hockey,sci.crypt,sci.electronics,sci.med,sci.space,soc.religion.christian,talk.politics.guns,talk.politics.mideast,talk.politics.misc,talk.religion.misc
alt.atheism,247,1,0,2,0,2,1,0,1,1,0,3,1,6,7,25,0,3,1,18
comp.graphics,1,304,16,7,5,21,3,2,1,2,1,5,10,0,5,1,0,2,0,3
comp.os.ms-windows.misc,0,17,289,36,10,11,4,1,0,5,0,3,1,2,5,2,1,1,1,5
comp.sys.ibm.pc.hardware,0,9,25,294,27,1,14,3,1,1,0,1,14,0,1,0,0,0,0,1
comp.sys.mac.hardware,0,5,5,17,333,0,9,0,0,2,1,1,7,1,0,0,2,0,2,0
comp.windows.x,1,33,39,3,3,299,3,1,2,1,0,2,1,2,3,1,1,0,0,0
misc.forsale,0,1,1,9,5,0,356,5,1,2,1,1,5,2,0,0,0,0,0,1
rec.autos,0,1,0,5,2,0,10,358,6,2,0,0,7,1,0,0,2,0,2,0
rec.motorcycles,0,0,0,1,0,0,4,9,380,1,0,0,0,2,0,1,0,0,0,0
rec.sport.baseball,0,0,1,0,1,0,4,0,0,378,12,0,0,0,0,0,0,1,0,0


Die Verwechslungen von `alt.atheism` mit `talk.religion.misc` und `soc.religion.christian` kann man in Nicht-Diagonal-Elementen der Confusion Matrix gut erkennen. Die könnten einem Menschen auch passieren, genau wie `comp.os.ms-windows.misc` mit `comp.sys.ibm.pc.hardware` und `comp.windows.x`

## Unsupervised Learning

`scikit-learn` kann nebem *überwachtem Lernen* auch viel Funktionen zum *unüberwachten Lernen* vorweisen. Unüberwachtes Lernen kannst du dazu nutzen, die organische Struktur eines Textkorpus zu explorieren:

In [7]:
from sklearn.decomposition import NMF
nmf = NMF(n_components=5)
nmf.fit(vectors)

NMF(n_components=5)

In [8]:
features = vectorizer.get_feature_names()
topics = []
for topic, word_vector in enumerate(nmf.components_):
    total = word_vector.sum()
    largest = word_vector.argsort()[::-1] # invert sort order
    
    topics.append([f" {features[largest[i]]}" for i in range(5)])
pd.DataFrame(topics, columns=[f"Wort {i}" for i in range(5)], index=[f"Topic {i}" for i in range(5)])

,Wort 0,Wort 1,Wort 2,Wort 3,Wort 4
Topic 0,the,of,and,to,in
Topic 1,the,to,for,it,and
Topic 2,you,to,that,is,it
Topic 3,he,the,was,his,and
Topic 4,edu,university,cs,posting,article


Hier siehst du, dass wir noch mehr Arbeit in die Vorbereitung der Daten stecken müssen. Die wichtigsten Worte der Topics sind fast alle ziemlich bedeutungslos (sog. *Stoppworte*). Lösungen dafür schauen wir uns im Laufe des Kurses an.

## `scikit-learn` kann noch viel mehr

Du hast bisher nur an der Oberfläche von `scikit-learn` gekratzt. `scikit-learn` bietet noch viel mehr Algorithmen,
Methoden und Konzepte, die wir uns in späteren Lektionen genau ansehen werden.